# Librerias

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text 
import re

# Conexion 

In [2]:
user = "root"
password = "Levp13aa"
host = "localhost"
database = "futbol_dw"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

## Jugadores 

In [22]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS silver_players;"))
    conn.commit()
print(" Tabla 'silver_players' eliminada.")

 Tabla 'silver_players' eliminada.


In [ ]:
import numpy as np
import pandas as pd

# ============================================================
# 1. Helpers: z-score y medias seguras
# ============================================================

def zscore_series(s: pd.Series) -> pd.Series:
    """Z-score: (x - mean)/std, ignorando NaNs. Si std=0 -> NaN."""
    s = pd.to_numeric(s, errors="coerce")
    m = s.mean()
    sd = s.std()
    if sd == 0 or np.isnan(sd):
        return pd.Series(np.nan, index=s.index)
    return (s - m) / sd

def add_zscore(df: pd.DataFrame,
               group_cols: list[str],
               value_cols: list[str],
               suffix: str) -> pd.DataFrame:
    """
    Para cada columna en value_cols, calcula z-score dentro de cada grupo
    definido por group_cols y crea una nueva columna col+suffix.
    """
    df = df.copy()
    for col in value_cols:
        if col not in df.columns:
            continue
        zname = f"{col}{suffix}"
        df[zname] = (
            df.groupby(group_cols)[col]
              .transform(zscore_series)
        )
    return df

def safe_mean(df: pd.DataFrame, cols: list[str]) -> pd.Series:
    """
    Media por fila ignorando:
      - columnas que no existan
      - columnas None
    Si para una fila todas las columnas son NaN -> NaN.
    """
    valid_cols = [c for c in cols if c is not None and c in df.columns]
    if not valid_cols:
        return pd.Series(np.nan, index=df.index)
    return df[valid_cols].mean(axis=1, skipna=True)

def zcol(base_name: str) -> str | None:
    """
    Dado un nombre base (por ejemplo 'goals_per_90'),
    intenta usar primero el z-score por posición (_z_pos),
    luego por competición (_z_comp). Si no existe, devuelve None.
    """
    cand_pos = f"{base_name}_z_pos"
    cand_comp = f"{base_name}_z_comp"
    if cand_pos in dfp.columns:
        return cand_pos
    if cand_comp in dfp.columns:
        return cand_comp
    return None

# ============================================================
# 2. Cargar bronze_players y renombrar columnas
# ============================================================

dfp = pd.read_sql("SELECT * FROM bronze_players;", engine)

# Mapeo basado en LAS COLUMNAS REALES de bronze_players
player_map_bronze_to_sql = {
    "Rk": "player_rank",
    "Date": "match_date",
    "Comp": "competition",
    "Age": "player_age",
    "Nation": "nation",
    "Start": "is_starter",
    "Min": "minutes_played",
    "Player": "player_name",
    "Team": "team_fbref",
    "Team_1": "team_fbref_alt",

    # Goles y asistencias
    "Performance_Gls": "goals",
    "Performance_Ast": "assists",
    "Performance_G_A": "goals_plus_assists",
    "Performance_G_PK": "goals_non_penalty",
    "Performance_PK": "penalty_goals",
    "Performance_PKatt": "penalty_attempts",
    "Performance_PKm": "penalty_missed",

    # Tiros (usamos la tabla Standard_)
    "Standard_Sh": "shots_total",
    "Standard_SoT": "shots_on_target",
    "Standard_G_Sh": "goals_per_shot",
    "Standard_G_SoT": "goals_per_shot_on_target",
    "Standard_Dist": "shot_avg_distance",
    "Standard_FK": "shots_free_kicks",

    # Pases (tabla Total_)
    "Total_Att": "passes_attempted",
    "Total_Cmp": "passes_completed",
    "Total_KP": "key_passes",
    "Total_PPA": "passes_into_penalty_area",
    "Total_CrsPA": "crosses_into_penalty_area",
    "Total_PrgP": "progressive_passes",
    "Total_TotDist": "passes_total_distance",
    "Total_PrgDist": "passes_progressive_distance",

    # SCA/GCA totales (por si quieres usarlos después)
    "SCA": "sca",
    "GCA": "gca",

    "SCA_Types_PassLive": "passes_live_shot",
    "SCA_Types_PassDead": "passes_dead_shot",
    "SCA_Types_TO": "take_ons_shot",
    "SCA_Types_Sh": "shots_lead_shot",
    "SCA_Types_Fld": "fouls_drawn_shot",
    "SCA_Types_Def": "defensive_actions_shot",

    "GCA_Types_PassLive": "passes_live_goal",
    "GCA_Types_PassDead": "passes_dead_goal",
    "GCA_Types_TO": "take_ons_goal",
    "GCA_Types_Sh": "shots_lead_goal",
    "GCA_Types_Fld": "fouls_drawn_goal",
    "GCA_Types_Def": "defensive_actions_goal",

    # Toques
    "Touches_Touches": "touches_total",
    "Touches_Def_Pen": "touches_def_pen_area",
    "Touches_Def_3rd": "touches_def_third",
    "Touches_Mid_3rd": "touches_mid_third",
    "Touches_Att_3rd": "touches_att_third",
    "Touches_Att_Pen": "touches_att_pen_area",
    "Touches_Live": "touches_live",

    # Defensa
    "Tackles_Tkl": "tackles_total",
    "Tackles_TklW": "tackles_won",
    "Tackles_Def_3rd": "tackles_def_third",
    "Tackles_Mid_3rd": "tackles_mid_third",
    "Tackles_Att_3rd": "tackles_att_third",

    "Blocks_Blocks": "blocks_total",
    "Blocks_Sh": "blocks_shot",
    "Blocks_Pass": "blocks_pass",
    "Blocks_Int": "interceptions",
    "Blocks_Tkl_Int": "tackles_plus_interceptions",
    "Blocks_Clr": "clearances",
    "Blocks_Err": "errors_lead_shot",

    # Faltas y tarjetas
    "CrdY": "yellow_cards",
    "Performance_CrdR": "red_cards",
    "Performance_2CrdY": "second_yellow_cards",
    "Performance_Fls": "fouls_committed",
    "Performance_Fld": "fouls_drawn",

    # Posición
    "Performance_Pos": "position_raw",
}

# Si tuviste clean_column_name en Bronze, lo usamos igual aquí
player_clean_map = {clean_column_name(k): v for k, v in player_map_bronze_to_sql.items()}
dfp = dfp.rename(columns=player_clean_map)

# ============================================================
# 3. Limpieza básica: nombres, posición, minutos
# ============================================================

# Nombre jugador y equipo limpio
if "player_name" in dfp.columns:
    dfp["player_name_clean"] = dfp["player_name"].astype(str).str.lower().str.strip()
elif "Player" in dfp.columns:
    dfp["player_name_clean"] = dfp["Player"].astype(str).str.lower().str.strip()

dfp["team_name_clean"] = dfp["team_name"].astype(str).str.lower().str.strip()

# Minutos
if "minutes_played" in dfp.columns:
    dfp["minutes_played"] = pd.to_numeric(dfp["minutes_played"], errors="coerce")

# Posición: usar solo lo que va antes de la coma ("FW,MF" -> "FW")
pos_col = None
for cand in ["position_raw", "position", "Pos", "pos", "Performance_Pos"]:
    if cand in dfp.columns:
        pos_col = cand
        break

print("Columna de posición detectada (raw):", pos_col)

if pos_col is not None:
    dfp["position_clean"] = (
        dfp[pos_col]
        .astype(str)
        .str.split(",")
        .str[0]
        .str.strip()
    )
else:
    dfp["position_clean"] = np.nan

# ============================================================
# 4. Métricas por 90 minutos
# ============================================================

per90_base_cols = [
    "goals",
    "assists",
    "goals_plus_assists",
    "shots_total",
    "shots_on_target",
    "key_passes",
    "passes_attempted",
    "passes_completed",
    "progressive_passes",
    "touches_total",
    "tackles_total",
    "interceptions",
    "blocks_total",
    "clearances",
    "yellow_cards",
    "red_cards",
    "fouls_committed",
]

for col in per90_base_cols:
    if col in dfp.columns and "minutes_played" in dfp.columns:
        new_col = f"{col}_per_90"
        dfp[new_col] = np.where(
            dfp["minutes_played"] > 0,
            dfp[col] * 90 / dfp["minutes_played"],
            np.nan
        )

# Ratios adicionales
if "shots_total" in dfp.columns and "shots_on_target" in dfp.columns:
    dfp["shots_on_target_pct"] = np.where(
        dfp["shots_total"] > 0,
        dfp["shots_on_target"] / dfp["shots_total"],
        np.nan
    )

if "passes_attempted" in dfp.columns and "passes_completed" in dfp.columns:
    dfp["passes_completion_pct"] = np.where(
        dfp["passes_attempted"] > 0,
        dfp["passes_completed"] / dfp["passes_attempted"],
        np.nan
    )

# ============================================================
# 5. Columnas a normalizar (z-score)
# ============================================================

norm_cols = [c for c in dfp.columns if c.endswith("_per_90")]

extra_norm_cols = [
    "goals_per_shot",
    "goals_per_shot_on_target",
    "shots_on_target_pct",
    "passes_completion_pct",
]

for c in extra_norm_cols:
    if c in dfp.columns:
        norm_cols.append(c)

# ============================================================
# 6. Z-score por competición, temporada y posición
# ============================================================

if "competition" in dfp.columns and "season" in dfp.columns:
    # nivel de liga/comp
    dfp = add_zscore(dfp, ["competition", "season"], norm_cols, "_z_comp")

    # por posición dentro de competición
    if "position_clean" in dfp.columns:
        dfp = add_zscore(dfp, ["competition", "season", "position_clean"], norm_cols, "_z_pos")

# ============================================================
# 7. Construir índices compuestos de estilo
# ============================================================

# Finishing
dfp["idx_finishing"] = safe_mean(dfp, [
    zcol("goals_per_90"),
    zcol("goals_per_shot"),
    zcol("goals_per_shot_on_target"),
    zcol("shots_on_target_pct"),
])

# Playmaking (generación de ocasiones)
dfp["idx_playmaking"] = safe_mean(dfp, [
    zcol("key_passes_per_90"),
    zcol("passes_into_penalty_area_per_90"),
    zcol("crosses_into_penalty_area_per_90"),
    zcol("progressive_passes_per_90"),
])

# Progresión de balón
dfp["idx_progression"] = safe_mean(dfp, [
    zcol("progressive_passes_per_90"),
    zcol("touches_att_third_per_90"),
    zcol("touches_att_pen_area_per_90"),
])

# Participación en juego
dfp["idx_involvement"] = safe_mean(dfp, [
    zcol("touches_total_per_90"),
    zcol("passes_attempted_per_90"),
    zcol("passes_completed_per_90"),
])

# Defensa / presión
dfp["idx_defending"] = safe_mean(dfp, [
    zcol("tackles_total_per_90"),
    zcol("interceptions_per_90"),
    zcol("blocks_total_per_90"),
    zcol("clearances_per_90"),
])

# Disciplina (tarjetas y faltas) -> signo negativo: más tarjetas = peor índice
dfp["idx_discipline"] = -safe_mean(dfp, [
    zcol("yellow_cards_per_90"),
    zcol("red_cards_per_90"),
    zcol("fouls_committed_per_90"),
])

# ============================================================
# 8. Guardar a silver_players
# ============================================================

dfp.to_sql(
    "silver_players",
    engine,
    if_exists="replace",
    index=False,
    method="multi",
    chunksize=1000
)

print(f" Tabla 'silver_players' creada con {dfp.shape[0]} filas y {dfp.shape[1]} columnas.")

Columna de posición detectada (raw): position_raw
 Tabla 'silver_players' creada con 58452 filas y 164 columnas.

Resumen índices:
       idx_finishing  idx_playmaking  idx_progression  idx_involvement  \
count   54631.000000    5.842500e+04     5.842500e+04     5.845200e+04   
mean       -0.090240    1.520203e-17     3.721456e-17    -3.209185e-17   
std         0.596889    7.646271e-01     9.997432e-01     9.668673e-01   
min        -0.665559   -9.786818e-01    -1.281627e+00    -2.435711e+00   
25%        -0.374119   -3.581056e-01    -4.437489e-01    -4.907473e-01   
50%        -0.178952   -1.487934e-01    -1.736367e-01    -1.687292e-01   
75%        -0.123882    1.290873e-01     1.679328e-01     2.669980e-01   
max        20.234513    2.847583e+01     3.843219e+01     3.458199e+01   

       idx_defending  idx_discipline  
count   5.845200e+04    5.845200e+04  
mean    9.117003e-19    2.309641e-18  
std     5.451798e-01    6.777657e-01  
min    -7.561043e-01   -4.011318e+01  
25%    

## Equipos 

In [25]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS silver_teams;"))
    conn.commit()
print(" Tabla 'silver_teams' eliminada.")

 Tabla 'silver_teams' eliminada.


In [ ]:
import numpy as np
import pandas as pd


def zscore_series(s: pd.Series) -> pd.Series:
    """Z-score: (x - mean)/std, ignorando NaNs. Si std=0 -> NaN."""
    s = pd.to_numeric(s, errors="coerce")
    m = s.mean()
    sd = s.std()
    if sd == 0 or np.isnan(sd):
        return pd.Series(np.nan, index=s.index)
    return (s - m) / sd

def add_zscore(df: pd.DataFrame,
               group_cols: list[str],
               value_cols: list[str],
               suffix: str) -> pd.DataFrame:
    """
    Para cada columna en value_cols, calcula z-score dentro de cada grupo
    definido por group_cols y crea una nueva columna col+suffix.
    """
    df = df.copy()
    for col in value_cols:
        if col not in df.columns:
            continue
        zname = f"{col}{suffix}"
        df[zname] = (
            df.groupby(group_cols)[col]
              .transform(zscore_series)
        )
    return df

def safe_mean(df: pd.DataFrame, cols: list[str], default=np.nan) -> pd.Series:
    """
    Media por fila ignorando:
      - columnas que no existan
      - columnas None

    Si para una fila todas las columnas son NaN -> default.
    """
    valid_cols = [c for c in cols if c is not None and c in df.columns]
    if not valid_cols:
        return pd.Series(default, index=df.index)
    s = df[valid_cols].mean(axis=1, skipna=True)
    if default is not np.nan:
        s = s.fillna(default)
    return s

# ============================================================
# 2. Diccionario de mapeo y carga desde bronze_teams
# ============================================================

team_map_original_to_sql = {
    "Rk": "team_rank",
    "Date": "match_date",
    "Comp": "competition",
    "Result": "result",
    "GF": "goals_for",
    "GA": "goals_against",
    "Poss": "possession_pct",
    # métricas que sí existen en tu schema:
    "Touches": "touches_total",
    "PrgP": "progressive_passes",
    "PrgR": "progressive_receptions",
    "Passes Att": "passes_attempted",
    "Passes Cmp": "passes_completed",
    "Passes Cmp%": "passes_completion_pct",
    "KP": "key_passes",
    "1/3": "passes_final_third",
    "PPA": "passes_into_penalty_area",
    "CrsPA": "crosses_into_penalty_area",
    "Tkl": "tackles_total",
    "Blocks": "blocks_total",
    "Int": "interceptions",
    "Clr": "clearances",
    "CrdY": "yellow_cards",
    "CrdR": "red_cards",
    "Fls": "fouls_committed",
    "Fld": "fouls_drawn",
}

team_clean_map = {clean_column_name(k): v for k, v in team_map_original_to_sql.items()}

# Cargar Bronze
dft = pd.read_sql("SELECT * FROM bronze_teams;", engine)

# Renombrar columnas que hagan match con el diccionario
dft = dft.rename(columns=team_clean_map)

# ============================================================
# 3. Limpieza de porcentajes (posesión, etc.)
# ============================================================

percent_cols = [
    "possession_pct",
    "shots_on_target_pct",
    "passes_completion_pct",
]

for col in percent_cols:
    if col in dft.columns:
        dft[col] = (
            dft[col]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", ".", regex=False)
        )
        dft[col] = pd.to_numeric(dft[col], errors="coerce")

# ============================================================
# 4. Definir columnas numéricas que SÍ existen (o pueden existir)
#    Nos apoyamos en tu schema real
# ============================================================

numeric_cols = [
    "goals_for", "goals_against",          # GF, GA
    "touches_total",                       # Touches -> touches_total
    "tackles_total",                       # Tkl -> tackles_total (si existe)
    "yellow_cards", "red_cards",           # CrdY, CrdR
    "fouls_committed", "fouls_drawn",      # Fls, Fld
    # uso de SCA/GCA totales (en tu schema existen SCA y GCA):
    "SCA", "GCA",
]

# Extra: columnas de verticalidad / volumen de pase que sí tienes
vertical_extra_cols = [
    "Total_PrgP",      # pases progresivos totales
    "Total_PrgDist",   # distancia progresiva
    "Total_TotDist",   # distancia total de pases
    "Total_PPA",       # pases al área
    "Total_CrsPA",     # centros al área
    "Touches_Att_3rd", # toques en tercio ofensivo
    "Touches_Att_Pen", # toques en área rival
    "Standard_Dist",   # distancia media de tiro
    "Total_Cmp",       # pases completados totales
    "Total_Att",       # pases intentados totales
]

numeric_cols = numeric_cols + vertical_extra_cols

# Asegurar numéricos
for col in numeric_cols:
    if col in dft.columns:
        dft[col] = pd.to_numeric(dft[col], errors="coerce")

# Columnas de porcentaje que realmente existan
team_norm_cols = []
for col in numeric_cols + percent_cols:
    if col in dft.columns:
        team_norm_cols.append(col)

team_norm_cols = list(sorted(set(team_norm_cols)))

# ============================================================
# 5. Z-scores por competición y temporada
# ============================================================

if "competition" in dft.columns and "season" in dft.columns:
    dft = add_zscore(dft, ["competition", "season"], team_norm_cols, "_z_comp")
else:
    print(" No se encontraron columnas competition y season; no se agregan z-scores.")

# Rellenar NaN de z-scores con 0 para no propagar NaN a los índices
for c in dft.columns:
    if c.endswith("_z_comp"):
        dft[c] = dft[c].fillna(0.0)

# ============================================================
# 6. Construir índices de estilo del equipo
# ============================================================

# Estilo de posesión / juego elaborado
dft["idx_possession_style"] = safe_mean(
    dft,
    [
        "possession_pct_z_comp",        # posesión
        "touches_total_z_comp",         # volumen de toques
        "Total_Cmp_z_comp",             # pases completados totales
        "Total_Att_z_comp",             # pases intentados totales
        "passes_completion_pct_z_comp", # si existiera
    ],
    default=0.0,
)

# Estilo ofensivo: capacidad de generar y convertir ocasiones
dft["idx_offensive_style"] = safe_mean(
    dft,
    [
        "goals_for_z_comp",     # GF
        "SCA_z_comp",           # acciones que generan tiro
        "GCA_z_comp",           # acciones que generan gol
        "Total_PPA_z_comp",     # pases al área
        "Total_CrsPA_z_comp",   # centros al área
    ],
    default=0.0,
)

# Estilo defensivo (solidez)
# Creamos explícitamente el negativo de goals_against_z_comp si existe
if "goals_against_z_comp" in dft.columns:
    dft["neg_goals_against_z_comp"] = -dft["goals_against_z_comp"]

dft["idx_defensive_style"] = safe_mean(
    dft,
    [
        "neg_goals_against_z_comp",  # menos goles en contra = mejor
        "tackles_total_z_comp",      # volumen de entradas
        "fouls_committed_z_comp",    # agresividad defensiva (puede ser buena o mala)
    ],
    default=0.0,
)

# Verticalidad / juego directo / progresión
vertical_z_cols = [
    c for c in [
        "Total_PrgP_z_comp",
        "Total_PrgDist_z_comp",
        "Total_TotDist_z_comp",
        "Total_PPA_z_comp",
        "Total_CrsPA_z_comp",
        "Touches_Att_3rd_z_comp",
        "Touches_Att_Pen_z_comp",
        "Standard_Dist_z_comp",
    ]
    if c in dft.columns
]

dft["idx_verticality_style"] = safe_mean(
    dft,
    vertical_z_cols,
    default=0.0,
)

# Agresividad / intensidad (entradas y faltas)
dft["idx_aggressiveness_style"] = safe_mean(
    dft,
    [
        "tackles_total_z_comp",
        "fouls_committed_z_comp",
        "yellow_cards_z_comp",
        "red_cards_z_comp",
    ],
    default=0.0,
)

# ============================================================
# 7. Guardar a silver_teams
# ============================================================

dft.to_sql(
    "silver_teams",
    engine,
    if_exists="replace",
    index=False,
    method="multi",
    chunksize=1000
)

print(f" Tabla 'silver_teams' creada con {dft.shape[0]} filas y {dft.shape[1]} columnas.")

# Chequeo rápido de que los índices NO son todos cero
print("\nResumen índices de equipos:")
print(
    dft[
        [
            "idx_possession_style",
            "idx_offensive_style",
            "idx_defensive_style",
            "idx_verticality_style",
            "idx_aggressiveness_style",
        ]
    ].describe()
)


 Tabla 'silver_teams' creada con 3708 filas y 135 columnas.

Resumen índices de equipos:
       idx_possession_style  idx_offensive_style  idx_defensive_style  \
count          3.708000e+03         3.708000e+03         3.708000e+03   
mean           1.149746e-17         1.916243e-17         1.700665e-17   
std            9.618301e-01         7.008322e-01         7.170119e-01   
min           -2.297753e+00        -1.426123e+00        -2.908505e+00   
25%           -7.011592e-01        -5.044197e-01        -4.575299e-01   
50%           -7.713201e-02        -7.499375e-02         5.185802e-03   
75%            6.026935e-01         4.070750e-01         4.703278e-01   
max            3.698749e+00         3.685791e+00         2.228709e+00   

       idx_verticality_style  idx_aggressiveness_style  
count           3.708000e+03              3.708000e+03  
mean            1.341370e-17              5.940352e-17  
std             6.372247e-01              7.222691e-01  
min            -1.774499e